In [1]:
import pandas as pd
from pydantic import BaseModel, validator
import numpy as np
from datetime import datetime
from enum import Enum
from collections.abc import Iterable
from functools import partial
df_result = pd.DataFrame()

In [2]:
df_ref = pd.read_csv('kidney_test.csv')

df_ref['model_id'] = '1'
df_ref['model_version'] = '2'
df_ref['period'] = 'reference'

In [3]:
df_ref.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pe,ane,class,y_pred,clf_target,y_pred_proba,pred_timestamp,model_id,model_version,period
0,214,55.0,80.0,1.010,3.0,1,1,0,1,1,...,1,0,1,1,0,1.00,2022-07-10 22:30:30,1,2,reference
1,372,28.0,60.0,1.025,0.0,0,1,1,0,0,...,0,0,0,1,0,1.00,2022-07-10 22:30:30,1,2,reference
2,367,60.0,80.0,1.025,0.0,0,1,1,0,0,...,0,0,0,0,0,0.99,2022-07-10 22:30:30,1,2,reference
3,260,35.0,80.0,1.020,0.0,0,1,1,0,0,...,0,0,0,0,0,0.98,2022-07-15 22:30:30,1,2,reference
4,374,61.0,70.0,1.025,0.0,0,1,1,0,0,...,0,0,0,1,0,0.91,2022-07-15 22:30:30,1,2,reference


In [4]:
from influxdb import DataFrameClient, InfluxDBClient

dbhost = "localhost"
dbport = 8086
dbuser = "admin"
dbpasswd = "pass123"
dbname = "testDB"
protocol = "line"

client = InfluxDBClient(dbhost, dbport, dbuser, dbpasswd, dbname)

query = "select * from test5Aug"

df_new = pd.DataFrame(client.query(query, chunked=True).get_points())

df_new.rename(columns = {'time': 'pred_timestamp'}, inplace = True)
df_new["pred_timestamp"]=pd.to_datetime(df_new["pred_timestamp"]).dt.strftime('%Y-%m-%d %H:%M:%S')
# df_new['pred_timestamp'] = datetime.strptime('20/8/2022 22:30:30','%d/%m/%Y %H:%M:%S')
# df_new['eval_timestamp'] = datetime(2022,8,26)
df_new['model_id'] = '1'
df_new['model_version'] = '2'

try:
    dbname = "testDB_period"
    client = InfluxDBClient(dbhost, dbport, dbuser, dbpasswd, dbname)
    query = "select * from testDB_period"
    df_period_end = pd.DataFrame(client.query(query, chunked=True).get_points())
    if (len(df_period_end) >0):
        df_new = df_new[df_new['pred_timestamp'] > max(df_period_end['time'])]
except:
    pass
    # TODO: Need to add InfluxDB connection exceptions here


In [5]:
df_new

,pred_timestamp,age,al,ane,appet,ba,bgr,bp,bu,cad,...,project_id,rbc,rbcc,sc,sg,sod,su,target,wbcc,y_name
0,2022-08-06 00:35:06,83.0,3.0,1,1,0,102.0,70.0,60.0,0,...,MPM,1,3.1,2.6,1.020,115.0,0,1,12800.0,None
1,2022-08-09 11:47:43,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,MPM,1,5.3,1.2,1.025,142.0,0,0,6900.0,None
2,2022-08-12 12:24:23,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,MPM,1,5.3,1.2,1.025,142.0,0,0,6900.0,None
3,2022-08-12 12:36:14,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,MPM,1,5.3,1.2,1.025,142.0,0,0,6900.0,None
4,2022-08-16 11:34:56,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,None,1,5.3,1.2,1.025,142.0,0,0,6900.0,ABC
5,2022-08-16 18:41:37,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,None,1,5.3,1.2,1.025,142.0,0,0,6900.0,ABC
6,2022-08-16 23:14:58,25.0,0.0,0,0,0,121.0,80.0,19.0,0,...,None,1,5.3,1.2,1.025,142.0,0,0,6900.0,ABC
7,2022-08-16 23:18:58,17.0,0.0,0,0,0,114.0,60.0,50.0,0,...,None,1,5.9,1.0,1.025,135.0,0,0,7200.0,ABC
8,2022-08-17 10:59:23,17.0,0.0,0,0,0,114.0,60.0,50.0,0,...,None,1,5.9,1.0,1.025,135.0,0,0,7200.0,ABC
9,2022-08-17 22:43:52,17.0,0.0,0,0,0,114.0,60.0,50.0,0,...,None,1,5.9,1.0,1.025,135.0,0,0,7200.0,ABC


In [6]:
from pulsar_metrics.metrics.base import MetricResults

In [7]:
result = MetricResults(name = "accuracy", type = "performance", model_id = "1", model_version = "1.0", period_end = datetime(2022,9,1), period_start = datetime(2022,7,1))

In [8]:
result

MetricResults(metric_name=None, type='performance', model_id='1', model_version='1.0', data_id=None, feature=None, value=None, status=None, threshold=None, period_start=datetime.datetime(2022, 7, 1, 0, 0), period_end=datetime.datetime(2022, 9, 1, 0, 0), eval_timestamp=datetime.datetime(2022, 9, 15, 1, 12, 56, 119578), conf_int=None)

In [9]:
from pulsar_metrics.metrics.base import MetricsType
MetricsType._member_names_

['performance', 'drift', 'custom']

In [10]:
from pulsar_metrics.metrics.performance import PerformanceMetricsFuncs
PerformanceMetricsFuncs._member_names_

['accuracy',
 'precision',
 'recall',
 'f1',
 'log_loss',
 'auc',
 'aucpr',
 'brier',
 'mse',
 'mae',
 'mape',
 'r2']

In [11]:
PerformanceMetricsFuncs['f1'].value(df_ref.y_pred, df_ref.clf_target, average = 'macro')

0.49579831932773105

In [12]:
from pulsar_metrics.metrics.performance import PerformanceMetric

precision = PerformanceMetric(name = 'precision', data = df_ref, y_name = 'clf_target')

In [13]:
precision._result

In [14]:
precision_result = precision.evaluate(bootstrap=False, average = 'micro', threshold = 0.5)

In [15]:
dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [16]:
# accuracy
precision = PerformanceMetric(name = 'accuracy', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [17]:
# recall
precision = PerformanceMetric(name = 'recall', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, average = 'micro', threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [18]:
# f1
precision = PerformanceMetric(name = 'f1', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, average = 'micro', threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [19]:
#brier
precision = PerformanceMetric(name = 'brier', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False,  threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [20]:
#mse
precision = PerformanceMetric(name = 'mse', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False,  threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [21]:
#mae
precision = PerformanceMetric(name = 'mae', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False,  threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [22]:
#mape
precision = PerformanceMetric(name = 'mape', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False,  threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [23]:
#r2
precision = PerformanceMetric(name = 'r2', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False,  threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [24]:
# log_loss
# this metric is not working with kidney disease test data
precision = PerformanceMetric(name = 'log_loss', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [25]:
#auc
# this metric is not working with kidney disease test data
precision = PerformanceMetric(name = 'auc', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [26]:
#aucpr
# this metric is not working with kidney disease test data
precision = PerformanceMetric(name = 'aucpr', data = df_ref, y_name = 'clf_target')
precision._result
precision_result = precision.evaluate(bootstrap=False, threshold = 0.5)


dict = vars(precision_result)
df_compute = pd.DataFrame([dict])
df_result = pd.concat([df_result, df_compute], ignore_index=True)

In [27]:
from pulsar_metrics.analyzers.base import Analyzer

In [28]:
analysis = Analyzer(name = 'First Analyzer', description='My first Analyzer', data = df_new)

In [29]:
analysis.add_drift_metrics(metrics_list=['wasserstein', 'ttest', 'ks_2samp','kl','manwu','levene','bftest','CvM'], 
features_list=['age','al','ane','appet','ba','bgr','bp','bu','cad','dm','hemo','htn','id','pc','pcc','pcv','pe','pot','rbc','rbcc','sg','sc','sod','su'])
# analysis.add_performance_metrics(metrics_list=['accuracy'], y_name = 'clf_target')

Drift metric 'wasserstein' for feature 'age' added to the analyzer list
Drift metric 'wasserstein' for feature 'al' added to the analyzer list
Drift metric 'wasserstein' for feature 'ane' added to the analyzer list
Drift metric 'wasserstein' for feature 'appet' added to the analyzer list
Drift metric 'wasserstein' for feature 'ba' added to the analyzer list
Drift metric 'wasserstein' for feature 'bgr' added to the analyzer list
Drift metric 'wasserstein' for feature 'bp' added to the analyzer list
Drift metric 'wasserstein' for feature 'bu' added to the analyzer list
Drift metric 'wasserstein' for feature 'cad' added to the analyzer list
Drift metric 'wasserstein' for feature 'dm' added to the analyzer list
Drift metric 'wasserstein' for feature 'hemo' added to the analyzer list
Drift metric 'wasserstein' for feature 'htn' added to the analyzer list
Drift metric 'wasserstein' for feature 'id' added to the analyzer list
Drift metric 'wasserstein' for feature 'pc' added to the analyzer l

In [30]:
analysis.run(data_ref = df_ref, options = {'ttest': {'alpha': 0.05, 'equal_var': False}})

/home/manuarya/Projects/pulsar-metrics/pulsar-metrics/.venv/lib/python3.9/site-packages/scipy/stats/morestats.py:2603: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom


In [31]:
analysis._results

[MetricResults(metric_name='wasserstein', type='drift', model_id='1', model_version='2', data_id=None, feature='age', value=23.666666666666668, status=None, threshold=None, period_start=Timestamp('2022-08-06 00:35:06'), period_end=Timestamp('2022-08-18 01:52:13'), eval_timestamp=datetime.datetime(2022, 9, 15, 1, 12, 56, 119578), conf_int=None),
 MetricResults(metric_name='wasserstein', type='drift', model_id='1', model_version='2', data_id=None, feature='al', value=0.33333333333333315, status=None, threshold=None, period_start=Timestamp('2022-08-06 00:35:06'), period_end=Timestamp('2022-08-18 01:52:13'), eval_timestamp=datetime.datetime(2022, 9, 15, 1, 12, 56, 119578), conf_int=None),
 MetricResults(metric_name='wasserstein', type='drift', model_id='1', model_version='2', data_id=None, feature='ane', value=0.0, status=None, threshold=None, period_start=Timestamp('2022-08-06 00:35:06'), period_end=Timestamp('2022-08-18 01:52:13'), eval_timestamp=datetime.datetime(2022, 9, 15, 1, 12, 56,

In [32]:
df_result_drift = analysis.results_to_pandas()

In [33]:
# df_result = pd.concat((df_result, df_result_drift), axis=1, ignore_index=True)
df_result = df_result.append(df_result_drift, ignore_index=True)

/tmp/ipykernel_20333/2230772446.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(df_result_drift, ignore_index=True)


In [34]:
df_result

,metric_name,type,model_id,model_version,data_id,feature,value,status,threshold,period_start,period_end,eval_timestamp,conf_int
0,precision,performance,1,2,None,prediction,0.583333,False,0.50,2022-07-10 22:30:30,2022-08-21 15:30:30,2022-09-15 01:12:56.119578,None
1,accuracy,performance,1,2,None,prediction,0.583333,False,0.50,2022-07-10 22:30:30,2022-08-21 15:30:30,2022-09-15 01:12:56.119578,None
2,recall,performance,1,2,None,prediction,0.583333,False,0.50,2022-07-10 22:30:30,2022-08-21 15:30:30,2022-09-15 01:12:56.119578,None
3,f1,performance,1,2,None,prediction,0.583333,False,0.50,2022-07-10 22:30:30,2022-08-21 15:30:30,2022-09-15 01:12:56.119578,None
4,brier,performance,1,2,None,prediction,0.416667,True,0.50,2022-07-10 22:30:30,2022-08-21 15:30:30,2022-09-15 01:12:56.119578,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,CvM,drift,1,2,None,rbcc,0.175061,False,0.05,2022-08-06 00:35:06,2022-08-18 01:52:13,2022-09-15 01:12:56.119578,None
200,CvM,drift,1,2,None,sg,0.000565,True,0.05,2022-08-06 00:35:06,2022-08-18 01:52:13,2022-09-15 01:12:56.119578,None
201,CvM,drift,1,2,None,sc,0.132167,False,0.05,2022-08-06 00:35:06,2022-08-18 01:52:13,2022-09-15 01:12:56.119578,None
202,CvM,drift,1,2,None,sod,0.064415,False,0.05,2022-08-06 00:35:06,2022-08-18 01:52:13,2022-09-15 01:12:56.119578,None


In [35]:
df_period = df_result.filter(['type','period_end'], axis=1)
df_period.set_index("period_end", inplace=True)
# df_period.loc[df_period["period_end"].idxmax()]

In [36]:
# df_period.to_csv('df_period.csv', index=False)
db_name = "testDB_period"
client = DataFrameClient(dbhost, dbport, dbuser, dbpasswd, db_name)
client.create_database(db_name)
client.get_list_database()
client.switch_database(db_name)

client.write_points(df_period, db_name, protocol='line', time_precision=None)

True